In [4]:
# coding: utf-8
import sys, os, time
print(os.getcwd())
current_dir = os.path.dirname(os.getcwd())
print(current_dir)
os.chdir(current_dir)

import numpy as np
import pickle
from dataset.mnist import load_mnist
from common.functions import sigmoid, softmax

def get_data():
    (x_train, t_train), (x_test, t_test) = load_mnist(flatten = True, normalize=True, one_hot_label=False)
    return x_test, t_test

def init_network():
    print(os.getcwd())
    with open('ch02/sample_weight.pkl','rb') as f:
        network = pickle.load(f)
    return network

def predict(network, x):
    W1,W2,W3 = network['W1'], network['W2'], network['W3']
    b1,b2,b3 = network['b1'], network['b2'], network['b3']

    a1 = np.dot(x,W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1,W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2,W3) + b3
    y = softmax(a3)

    return y

x,t = get_data()
network = init_network()
accuracy_cnt = 0
for i in range(len(x)):
    y = predict(network, x[i])
    p = np.argmax(y)
    if p != t[i]:
        print('Predict num=',p,' Original num=',t[i])
        time.sleep(0.5)
        accuracy_cnt += 1
print('Accuracy:' + str(float(accuracy_cnt)/len(x)))


/workspaces/DL-Excersize
/workspaces
/workspaces


FileNotFoundError: [Errno 2] No such file or directory: 'ch02/sample_weight.pkl'

# MNIST 3층 신경망 추론 코드 설명

## 1. 전체 목적

이 코드는 **학습이 완료된 3층 신경망을 사용하여 MNIST 숫자 이미지의 분류 정확도를 측정**하는 프로그램이다.
학습은 이미 끝난 상태이며(`sample_weight.pkl`), 이 코드는 **추론(inference)만 수행**한다.

---

## 2. 전체 흐름

1. MNIST 테스트 데이터 로드
2. 저장된 가중치 로드
3. 순전파(forward propagation) 수행
4. 예측값과 정답 비교
5. 정확도 계산

---

## 3. 데이터 로드

```python
def get_data():
    (x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=True, one_hot_label=False)
    return x_test, t_test
```

* `flatten=True` → 28×28 이미지를 784차원 벡터로 변환
* `normalize=True` → 픽셀값을 0~1로 스케일링
* `one_hot_label=False` → 정답은 숫자 정수 (0~9)

결과:

* `x_test.shape = (10000, 784)`
* `t_test.shape = (10000,)`

---

## 4. 학습된 가중치 로드

```python
with open('ch02/sample_weight.pkl','rb') as f:
    network = pickle.load(f)
```

network는 딕셔너리:

* W1, W2, W3
* b1, b2, b3

구조는 다음과 같다:

784 → 50 → 100 → 10

---

## 5. 핵심: 순전파 과정

```python

a1 = np.dot(x,W1) + b1
z1 = sigmoid(a1)

a2 = np.dot(z1,W2) + b2
z2 = sigmoid(a2)

a3 = np.dot(z2,W3) + b3
y = softmax(a3)
```

### (1) 1층

x: (784)
W1: (784, 50)

→ 결과: (50)

의미:
입력 픽셀 784개를 50개의 특징으로 변환

---

### (2) 2층

50 → 100 변환

더 추상적인 특징 생성

---

### (3) 출력층

100 → 10

각 숫자(0~9)에 대한 점수 계산 후
softmax로 확률화

---

## 6. 예측

```python
p = np.argmax(y)
```

* softmax 출력 중 가장 큰 확률의 인덱스 선택
* 그 인덱스가 예측 숫자

---

## 7. 정확도 계산

```python
if p == t[i]:
    accuracy_cnt += 1
```

전체 데이터에 대해 반복 후

Accuracy = 맞은 개수 / 전체 개수

---

## 8. 구조 요약

입력(784)
↓
선형변환 + sigmoid
↓
선형변환 + sigmoid
↓
선형변환 + softmax
↓
argmax → 예측 숫자

---

## 9. 중요한 포인트

* 이 코드는 학습이 아님 (역전파 없음)
* 순전파만 수행
* sigmoid는 은닉층
* softmax는 출력층
* 행렬곱은 다대다 가중합의 벡터화 표현

---

## 10. 한 문장 정리

이 코드는 "학습된 3층 완전연결 신경망이 MNIST 이미지를 확률적으로 분류하는 과정"을 순전파 방식으로 구현한 것이다.
